# First Sample

In [895]:
def get_financials(ticker="IDXX"):
    stock = yf.Ticker(ticker)
    income_statement = stock.income_stmt[::-1]  # Oldest first
    balance_sheet = stock.balance_sheet[::-1]
    cash_flow = stock.cashflow[::-1]
    
    return income_statement, balance_sheet, cash_flow

# Example usage
income_df, balance_df, cashflow_df = get_financials()

In [897]:
def get_financial_ratios(income_stmt, balance_sheet, cashflow_df):
    # Extract relevant data
    net_income = income_stmt.loc["Net Income", :].dropna()
    revenue = income_stmt.loc["Total Revenue", :].dropna()
    equity = balance_sheet.loc["Stockholders Equity", :].dropna()
    ebit = income_stmt.loc["EBIT", :].dropna()
    assets = balance_sheet.loc["Total Assets", :].dropna()
    liabilities = balance_sheet.loc["Total Liabilities Net Minority Interest", :].dropna()
    current_assets = balance_sheet.loc["Current Assets", :].dropna()
    current_liabilities = balance_sheet.loc["Current Liabilities", :].dropna()
    eps = income_stmt.loc["Diluted EPS", :].dropna()
    cash = balance_sheet.loc["Cash And Cash Equivalents", :].dropna()
    cash_flow_ops = cashflow_df.loc["Cash Flow From Continuing Operating Activities", :].dropna()
    gross_profit = income_stmt.loc["Gross Profit", :].dropna()

    stock = yf.Ticker("KO")
    latest_price = stock.history(period="1d")["Close"].iloc[-1]

    # Calculate ratios
    ratios = pd.Series(dtype="float64")
        
    ratios["Earnings 3Y Growth"] = (net_income.iloc[0] / net_income.iloc[-1]) - 1 if len(net_income) >= 3 else None
    ratios["Earnings 1Y Growth"] = (net_income.iloc[0] / net_income.iloc[1]) - 1 if len(net_income) >= 2 else None
    ratios["Operating Revenue 3Y Growth"] = (revenue.iloc[0] / revenue.iloc[-1]) - 1 if len(revenue) >= 3 else None
    ratios["Operating Revenue 1Y Growth"] = (revenue.iloc[0] / revenue.iloc[1]) - 1 if len(revenue) >= 2 else None
    ratios["Gross Profit 3Y Growth"] = (gross_profit.iloc[0] / gross_profit.iloc[-1]) - 1 if len(gross_profit) >= 3 else None
    ratios["ROE"] = net_income.iloc[0] / equity.iloc[0] if len(equity) > 0 else None
    ratios["ROE 3Y Growth"] = ((net_income.iloc[0] / equity.iloc[0]) / (net_income.iloc[-1] / equity.iloc[-1])) - 1 if len(equity) >= 3 else None
    ratios["P/E Ratio"] = latest_price / eps.iloc[0] if len(eps) > 0 else None
    ratios["P/B Ratio"] = stock.info.get("priceToBook", None)
    ratios["Cash per Stock"] = cash.iloc[0] / stock.info["sharesOutstanding"] if len(cash) > 0 and "sharesOutstanding" in stock.info else None
    ratios["52-Week Low"] = stock.info.get("fiftyTwoWeekLow", None)
    ratios["52-Week High"] = stock.info.get("fiftyTwoWeekHigh", None)
    ratios["200-Day Average"] = stock.info.get("twoHundredDayAverage", None)
    ratios["Current Price"] = latest_price
    ratios["Current Dividend Yield"] = stock.info.get("dividendYield", None)

    return ratios

# Example usage
ratios_df = get_financial_ratios(income_df, balance_df, cashflow_df)


In [899]:
income_df

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Operating Revenue,3660953000.0,3367324000.0,3215360000.0,2706655000.0
Total Revenue,3660953000.0,3367324000.0,3215360000.0,2706655000.0
Cost Of Revenue,1470983000.0,1362986000.0,1325928000.0,1135615000.0
Gross Profit,2189970000.0,2004338000.0,1889432000.0,1571040000.0
Other Gand A,335825000.0,326248000.0,309660000.0,300832000.0
General And Administrative Expense,335825000.0,326248000.0,309660000.0,300832000.0
Selling And Marketing Expense,566066000.0,524505000.0,486735000.0,434435000.0
Selling General And Administration,901891000.0,850753000.0,796395000.0,735267000.0
Research And Development,190951000.0,254820000.0,161009000.0,141249000.0
Operating Expense,1092842000.0,1105573000.0,957404000.0,876516000.0


# Stock picker with sectors

In [503]:
import os
import pandas as pd
import yfinance as yf
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [905]:
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return table["Symbol"]

def get_financials(ticker):
    stock = yf.Ticker(ticker)
    return stock.income_stmt[::-1], stock.balance_sheet[::-1], stock.cashflow[::-1]

def get_financial_ratios(ticker, income_stmt, balance_sheet, cashflows_stmt):
    try:
        net_income = income_stmt.loc["Net Income", :].dropna()
        revenue = income_stmt.loc["Total Revenue", :].dropna()
        gross_profit = income_stmt.loc["Gross Profit", :].dropna()
        equity = balance_sheet.loc["Stockholders Equity", :].dropna()
        cash = balance_sheet.loc["Cash And Cash Equivalents", :].dropna()
        fcf = cashflows_stmt.loc["Free Cash Flow", :].dropna()

        stock = yf.Ticker(ticker)
        latest_price = stock.history(period="1d")["Close"].iloc[-1]
        eps = income_stmt.loc["Diluted EPS", :].dropna()

        ratios = pd.Series(dtype="float64")
        
        ratios["Earnings 3Y Growth"] = (net_income.iloc[0] / net_income.iloc[-1]) - 1 if len(net_income) >= 3 else None
        ratios["Earnings 1Y Growth"] = (net_income.iloc[0] / net_income.iloc[1]) - 1 if len(net_income) >= 2 else None
        ratios["Operating Revenue 3Y Growth"] = (revenue.iloc[0] / revenue.iloc[-1]) - 1 if len(revenue) >= 3 else None
        ratios["Operating Revenue 1Y Growth"] = (revenue.iloc[0] / revenue.iloc[1]) - 1 if len(revenue) >= 2 else None
        ratios["Free Cash Flow 3Y Growth"] = (fcf.iloc[0] / fcf.iloc[-1]) - 1 if len(net_income) >= 3 else None
        ratios["Free Cash Flow 1Y Growth"] = (fcf.iloc[0] / fcf.iloc[1]) - 1 if len(net_income) >= 2 else None
        ratios["Gross Profit 3Y Growth"] = (gross_profit.iloc[0] / gross_profit.iloc[-1]) - 1 if len(gross_profit) >= 3 else None
        ratios["ROE"] = net_income.iloc[0] / equity.iloc[0] if len(equity) > 0 else None
        ratios["ROE 3Y Growth"] = ((net_income.iloc[0] / equity.iloc[0]) / (net_income.iloc[-1] / equity.iloc[-1])) - 1 if len(equity) >= 3 else None
        ratios["P/E Ratio"] = latest_price / eps.iloc[0] if len(eps) > 0 else None
        ratios["P/B Ratio"] = stock.info.get("priceToBook", None)
        ratios["Current Dividend Yield"] = stock.info.get("dividendYield", None)
        ratios["Debt to Equity"] = stock.info.get("debtToEquity", None) / 100
        ratios["Cash per Stock"] = cash.iloc[0] / stock.info["sharesOutstanding"] if len(cash) > 0 and "sharesOutstanding" in stock.info else None
        ratios["52-Week Low"] = stock.info.get("fiftyTwoWeekLow", None)
        ratios["52-Week High"] = stock.info.get("fiftyTwoWeekHigh", None)
        ratios["200-Day Average"] = stock.info.get("twoHundredDayAverage", None)
        ratios["Current Price"] = latest_price
        
    except Exception as e:
        # print(f"Error processing {ticker}: {e}")
        return None

    return ratios

def process_sp500():
    if os.path.exists("a.xlsx"):
        print("Loading existing data from output.xlsx...")
        ratios_df = pd.read_excel("output.xlsx", index_col=0)
        # tickers_df = get_sp500_tickers()
    else:
        tickers_df = get_sp500_tickers()
        data = {}

        for ticker in tickers_df:
            print(ticker)
            income_stmt, balance_sheet, cashflow_stmt = get_financials(ticker)
            ratios = get_financial_ratios(ticker, income_stmt, balance_sheet, cashflow_stmt)
            if ratios is not None:
                ratios["Sector"] = yf.Ticker(ticker).info.get("sector")  # Add sector info
                data[ticker] = ratios

        ratios_df = pd.DataFrame(data).T
        ratios_df.to_excel("output.xlsx")

    selected_stocks = {}
    selected_stocks_list = []

    for sector in ratios_df["Sector"].unique():
        sector_df = ratios_df[ratios_df["Sector"] == sector]
        numeric_sector_df = sector_df.drop(columns=["Sector"])  # Exclude non-numeric columns
        averages = numeric_sector_df.median()
    
        filtered = sector_df[
            (sector_df["Earnings 3Y Growth"] > 0) &
            (sector_df["Earnings 1Y Growth"] > 0) &
            (sector_df["Operating Revenue 1Y Growth"] > 0) &
            (sector_df["Operating Revenue 3Y Growth"] > 0) &
            (sector_df["Gross Profit 3Y Growth"] > 0)
        ]
    
        # Add sector as a column for easy identification
        filtered["Sector"] = sector
        selected_stocks_list.append(filtered)
    
    # Combine all selected stocks into a single DataFrame
    selected_stocks_df = pd.concat(selected_stocks_list)
    
    # Set a multi-index for better visualization
    selected_stocks_df.set_index(["Sector", selected_stocks_df.index], inplace=True)

    print("Sector-based filtering complete.")
    return selected_stocks_df

# Example usage
selected_stocks = process_sp500()
selected_stocks.to_excel("Filtered Stocks.xlsx")
print("Successfully exported Excel")

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
APO
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BAX
BDX
BRK.B
BBY
TECH
BIIB
BLK
BX
BK
BA
BKNG
BWA
BSX
BMY
AVGO
BR
BRO
BF.B
BLDR
BG
BXP
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DELL
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ERIE
ESS
EL
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUBB
HUM

PermissionError: [Errno 13] Permission denied: 'Filtered Stocks.xlsx'

In [891]:
selected_stocks

Earnings 3Y Growth Earnings 1Y Growth  \
Sector                                                               
Industrials            AOS             0.613801           1.361476   
                       ALLE            0.719376           0.179913   
                       AME             0.505192           0.132506   
                       CAT             2.447298           0.541387   
                       CTAS            0.414615           0.165861   
                       CPRT            0.455448           0.101216   
                       ETN              1.28227           0.307067   
                       FAST             0.34443           0.062655   
                       GPN             0.687253           7.845694   
                       HON              0.18393           0.139348   
                       HWM             1.931034            0.63113   
                       HUBB             1.16344            0.39183   
                       IEX             0.577819           0.015676   
                       LII             0.656189           0.187085   
                       PCAR            2.535813           0.527693   
                       PH              0.628896           0.365484   
                       PWR              0.67122           0.516095   
                       RTX             0.235507            0.49421   
                       RSG             0.789702           0.163619   
                       SNA               0.6126           0.109027   
                       TXT             1.980583           0.069686   
                       TT              1.367411           0.152235   
                       URI             0.857864           0.062294   
                       GWW             1.631655           0.182288   
                       WAB             0.968599            0.28752   
                       WM              0.540107           0.029491   
                       XYL             1.397638           0.715493   
Healthcare             CNC             0.494469            1.24792   
                       CVS             0.162279           0.935514   
                       DXCM            0.097042           0.587046   
                       IDXX            0.452521           0.244376   
                       PODD           29.338235          43.847826   
                       IQV             3.867384            0.24473   
                       JNJ             1.389085           0.959367   
                       RMD             1.151613           0.137479   
                       SYK             0.979362           0.342239   
                       UNH             0.453029           0.112376   
                       VRTX            0.334808           0.089585   
                       WST             0.714038           0.012801   
Technology             ACN               0.2299           0.057226   
                       ADBE            0.153049           0.024318   
                       AMAT             0.21892            0.04682   
                       ANET            2.289415           0.543367   
                       ADP              0.44391           0.099648   
                       BR              0.275068           0.107041   
                       CDNS            0.762727           0.226387   
                       CPAY            0.394302           0.028882   
                       FFIV            0.711074            0.43507   
                       FI              2.202505           0.212648   
                       FTNT            1.349642           0.338855   
                       GRMN            0.299612           0.324626   
                       IT              2.308276           0.092433   
                       IBM             0.342039            3.57439   
                       JKHY            0.225856           0.041375   
                       PAYX            0.540228           0.085468   
                       PAYC            

In [937]:
import yfinance as yf

tickers = a
stocks = yf.Ticker(" ".join(tickers))

# Access individual stock data
# aapl = stocks.tickers["AAPL"]
# msft = stocks.tickers["MSFT"]

# Get current price for all tickers
# prices = {ticker: stocks.tickers[ticker].history(period="1d")["Close"].iloc[-1] for ticker in tickers}
print(stocks)


yfinance.Ticker object <MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA APO AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BAX BDX BRK.B BBY TECH BIIB BLK BX BK BA BKNG BWA BSX BMY AVGO BR BRO BF.B BLDR BG BXP CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CAT CBOE CBRE CDW CE COR CNC CNP CF CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CAG COP ED STZ CEG COO CPRT GLW CPAY CTVA CSGP COST CTRA CRWD CCI CSX CMI CVS DHR DRI DVA DAY DECK DE DELL DAL DVN DXCM FANG DLR DFS DG DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ERIE ESS EL EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GE GEHC GEV GEN GNRC GD GIS GM GPC GILD GPN GL GDDY GS HAL HIG HAS HCA DOC HSIC HSY HES HPE HLT HOLX HD HON 

In [933]:
stocks.info

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MMM%20AOS%20ABT%20ABBV%20ACN%20ADBE%20AMD%20AES%20AFL%20A%20APD%20ABNB%20AKAM%20ALB%20ARE%20ALGN%20ALLE%20LNT%20ALL%20GOOGL%20GOOG%20MO%20AMZN%20AMCR%20AEE%20AEP%20AXP%20AIG%20AMT%20AWK%20AMP%20AME%20AMGN%20APH%20ADI%20ANSS%20AON%20APA%20APO%20AAPL%20AMAT%20APTV%20ACGL%20ADM%20ANET%20AJG%20AIZ%20T%20ATO%20ADSK%20ADP%20AZO%20AVB%20AVY%20AXON%20BKR%20BALL%20BAC%20BAX%20BDX%20BRK.B%20BBY%20TECH%20BIIB%20BLK%20BX%20BK%20BA%20BKNG%20BWA%20BSX%20BMY%20AVGO%20BR%20BRO%20BF.B%20BLDR%20BG%20BXP%20CHRW%20CDNS%20CZR%20CPT%20CPB%20COF%20CAH%20KMX%20CCL%20CARR%20CAT%20CBOE%20CBRE%20CDW%20CE%20COR%20CNC%20CNP%20CF%20CRL%20SCHW%20CHTR%20CVX%20CMG%20CB%20CHD%20CI%20CINF%20CTAS%20CSCO%20C%20CFG%20CLX%20CME%20CMS%20KO%20CTSH%20CL%20CMCSA%20CAG%20COP%20ED%20STZ%20CEG%20COO%20CPRT%20GLW%20CPAY%20CTVA%20CSGP%20COST%20CTRA%20CRWD%20CCI%20CSX%20CMI%20CVS%20DHR%20DRI%20DVA%20DAY%20DECK%20DE%20DELL%20DAL%20DVN%20DXCM

{'trailingPegRatio': None}

In [919]:
a = list(get_sp500_tickers())

# Calculate Portfolio returns

In [667]:
def get_prices(tickers, date):
    prices = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=date-timedelta(days=1), end=date)
        if not hist.empty:
            prices[ticker] = hist["Close"].iloc[0]
        else:
            prices[ticker] = None
    return prices

tickers = list(selected_stocks.index.get_level_values(1))

start_date = datetime.today().date() - timedelta(days=365)
end_date = datetime.today().date()

prices_start = get_prices(tickers, start_date)
prices_end = get_prices(tickers, end_date)

# Calculate individual stock returns
returns = [(prices_end[ticker] - prices_start[ticker]) / prices_start[ticker] for ticker in prices_start]

# Calculate the performance of the equally weighted portfolio (average return)
portfolio_return = np.mean(returns)
portfolio_return_percentage = portfolio_return * 100

portfolio_return_percentage

16.0531054760803

# Data Aggregate

In [769]:
def get_financials(ticker):
    stock = yf.Ticker(ticker)

    # Retrieve annual financials
    income_stmt = stock.income_stmt[::-1]
    balance_sheet = stock.balance_sheet[::-1]
    cashflow_stmt = stock.cashflow[::-1]

    # Check if the latest year is available
    latest_year = income_stmt.columns[0].year if not income_stmt.empty else None
    current_year = datetime.now().year - 1  # 2024 in this case

    if latest_year is None or latest_year < current_year:

        # Retrieve quarterly financials
        quarterly_income_stmt = stock.quarterly_financials
        quarterly_balance_sheet = stock.quarterly_balance_sheet
        quarterly_cashflow_stmt = stock.quarterly_cashflow

        # Filter 2024 data
        income_2024 = quarterly_income_stmt.loc[:, quarterly_income_stmt.columns.year == current_year]
        balance_2024 = quarterly_balance_sheet.loc[:, quarterly_balance_sheet.columns.year == current_year]
        cashflow_2024 = quarterly_cashflow_stmt.loc[:, quarterly_cashflow_stmt.columns.year == current_year]
        print(income_2024)

        # Ensure we have 4 quarters to sum
        if income_2024.shape[1] == 4 and cashflow_2024.shape[1] == 4:
            print(f"Aggregating quarterly data for {ticker} ({current_year})...")
            
            # Sum the quarterly data for 2024
            annual_income_2024 = income_2024.sum(axis=1)
            annual_cashflow_2024 = cashflow_2024.sum(axis=1)

            # Use the latest available quarterly balance sheet (not summed)
            annual_balance_2024 = balance_2024.iloc[:, 0] if balance_2024.shape[1] > 0 else None

            # Insert into annual statements
            income_stmt.insert(0, f"{current_year}-12-31", annual_income_2024)
            balance_sheet.insert(0, f"{current_year}-12-31", annual_balance_2024)
            cashflow_stmt.insert(0, f"{current_year}-12-31", annual_cashflow_2024)
            print("ALLA")

    return income_stmt, balance_sheet, cashflow_stmt


In [771]:
a, b, c = get_financials("CVS")

                                                       2024-09-30  \
Tax Effect Of Unusual Items                          -378756000.0   
Tax Rate For Calcs                                          0.324   
Normalized EBITDA                                    3187000000.0   
Total Unusual Items                                 -1169000000.0   
Total Unusual Items Excluding Goodwill              -1169000000.0   
Net Income From Continuing Operation Net Minori...     87000000.0   
Reconciled Depreciation                              1161000000.0   
Reconciled Cost Of Revenue                          82870000000.0   
EBITDA                                               2018000000.0   
EBIT                                                  857000000.0   
Net Interest Income                                  -752000000.0   
Interest Expense                                      752000000.0   
Normalized Income                                     877244000.0   
Net Income From Continuing And Dis